## Creating and solving your first model with Gurobi

### Seating Arrangement in Covid

- Owners of a movie theater must ensure that they can safely seat customers at minimum 2 meters of distance, while trying to seat as many people as possible to maximize their profit.Customers often visit a stadium show in groups. Members of a group must be seated together, but spectators in different groups must be seated at a safe distance away from other groups. 
- We will approach this issue as a bin packing problem. Each group is treated as a ‘box’ with a buffer zone of one seat surrounding the group on all sides. Each of these‘augmented groups’ (groups with buffer space) can now be packed adjacent to each other to maximize thegroups seated in a stadium. 
- We define an augmented row to be 3 ‘theater rows’ (the row in which the group is seated plus the buffer row above and below that row). 
- The total number of augmented rows in the stadium to be (the total number of regular rows/3). 
- Groups can only be seated in a single row (not in multiple rows), so the maximum allowed size of an augmented group is less than or equal to  W, the number of seats in one row. 
- d is the demand (people/groups who want to be seated is known ahead of time). Out of these, some will be seated, and some will not be seated.

#### Intro to our Model:
- Cineplex is considering seating groups, indexed i \in I in an theatre with rows j \in J and the width of the rows to contain W seats.
- Each group was a width in number of seats, w. 
- Each row can contain up to W seats
- The objective is to maximize the total number of people seated, minimize the number of groups that were not seated and maximizing the distances between those sitting in the same row.













#### Step 0: Present your mathematical model

Decision Variables:

$x_{ij}$: if group $i$ is sitting in row $j$.

$y{j}$: the surplus of seats in row $j$

Parameters:

$u$: total seated groups

$z$: the number of unseated groups

$p$: the total number of people seated in the theatre

$w_i$: the width of a group i for all i in I

Sets:
I [1,2,3...N] and 
J [1,2,3...M]


$$\begin{array}{rll}
\text{Maximize} & \displaystyle \sum_{i= 1}^N y_j + 4p -2z\\
\text{subject to:} & \text{Constraint 1:} \displaystyle \sum_{i=1}^N\sum_{j= 1}^M (x_{ij} * w_i) = p, 
& \text{Constraint 2:}\displaystyle \sum_{i=1}^N\sum_{j= 1}^M x_{ij} =u,
& \text{Constraint 3:}\displaystyle \sum_{i=1}^N (x_{ij} * w_i) + y_j =w \quad \forall j\in J\\
& \text{Constraint 4:}\displaystyle \sum_{j= 1}^M x_{ij} \leq 1 \quad \forall i\in I\\
& \text{Constraint 5:}\displaystyle z + u = N
\end{array}$$

#### Step 1: Import gurobipy module

In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum

#### Step 2: Define your model

In [2]:
m = gp.Model()

Using license file /Library/gurobi903/gurobi.lic
Academic license - for non-commercial use only


#### Step 3: Define your sets

In [3]:
#number of groups
I=35

#number of rows 30
J=15

#### Step 4: Define your parameters

In [4]:
#cwd = os.getcwd() 
#for data4 in os.listdir(cwd):
#    def readFloat_csv(data4):
#        with open(data4, newline='') as f_input:
#            return [list(map(float, row)) for row in csv.reader(f_input)]

#myData = readFloat_csv('data4.csv')
#print(myData)

#Fixed Width of a row
W = 15


#groups widths
w = [2,4, 4, 2, 3, 4, 6, 2, 3, 4,2,4, 4, 2, 3, 4, 6, 2, 3, 4,2,4, 4, 2, 3, 4, 6, 2, 3, 4,2,4, 4, 2, 3]

#### Step 5: Define your decision variables

In [5]:
x={}
for i in range(I):
    for j in range(J):
        x[i,j] = m.addVar(vtype=GRB.BINARY, name="x_"+str(i)+str(j))

y={}
for j in range(J):
    y[j] = m.addVar(vtype=GRB.INTEGER,lb=0, name="y_"+str(j))
    
u=m.addVar(vtype=GRB.INTEGER,lb=0, ub=I,name="u")

z=m.addVar(vtype=GRB.INTEGER,lb=0, ub=I, name="z")

p=m.addVar(vtype=GRB.INTEGER,lb=0, name="p")


#### Step 6: Set your objective function

In [6]:
m.setObjective(sum(y[j] for j in range(J)) + 4*(p) -2*(z), GRB.MAXIMIZE)

#### Step 6: Add your constraints

In [7]:
m.addConstrs((sum(x[i,j]for j in range(J)) <=1) for i in range(I)) 
m.addConstrs((sum(x[i,j]*w[i] for i in range(I)) +y[j] <= W )for j in range(J)) 
m.addConstr(quicksum(x[i,j]for i in range(I) for j in range(J)) == u)
m.addConstr(quicksum(x[i,j]*w[i] for i in range(I) for j in range(J)) == p)
m.addConstr( u + z == I)

<gurobi.Constr *Awaiting Model Update*>

#### Step 7: Solve the model

In [8]:
#m.Params.TimeLimit=300 #(seconds) optional: sets a time limit for optimization only if you need to prematurely stop the solution procedure
m.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 53 rows, 543 columns and 2119 nonzeros
Model fingerprint: 0x78d3d452
Variable types: 0 continuous, 543 integer (525 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 155.0000000
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolved: 51 rows, 541 columns, 1591 nonzeros
Variable types: 0 continuous, 541 integer (525 binary)

Root relaxation: objective 5.760000e+02, 84 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  576.00000    0    6  155.00000  576.00000   272%     -    0s
H    0     0                     576.0000000  576.00000  0.00%     -    0s
     0     0  576.00000    0    6  576.00000  576.00000  0

#### Step 8: Print variable values  (The Messy Way)

In [9]:
for myVars in m.getVars():
    print('%s %g' % (myVars.varName, myVars.x))
    
#https://docs.python.org/2.4/lib/typesseq-strings.html

x_00 -0
x_01 -0
x_02 -0
x_03 -0
x_04 -0
x_05 -0
x_06 -0
x_07 -0
x_08 -0
x_09 -0
x_010 -0
x_011 -0
x_012 1
x_013 -0
x_014 -0
x_10 -0
x_11 -0
x_12 -0
x_13 -0
x_14 -0
x_15 -0
x_16 -0
x_17 -0
x_18 -0
x_19 -0
x_110 -0
x_111 -0
x_112 -0
x_113 -0
x_114 1
x_20 -0
x_21 -0
x_22 -0
x_23 -0
x_24 -0
x_25 -0
x_26 -0
x_27 -0
x_28 -0
x_29 -0
x_210 -0
x_211 -0
x_212 -0
x_213 -0
x_214 1
x_30 -0
x_31 -0
x_32 -0
x_33 -0
x_34 -0
x_35 -0
x_36 -0
x_37 -0
x_38 -0
x_39 -0
x_310 1
x_311 -0
x_312 -0
x_313 -0
x_314 -0
x_40 -0
x_41 -0
x_42 -0
x_43 -0
x_44 -0
x_45 -0
x_46 -0
x_47 -0
x_48 -0
x_49 -0
x_410 -0
x_411 -0
x_412 -0
x_413 1
x_414 -0
x_50 -0
x_51 -0
x_52 -0
x_53 -0
x_54 -0
x_55 -0
x_56 -0
x_57 -0
x_58 -0
x_59 -0
x_510 -0
x_511 -0
x_512 -0
x_513 -0
x_514 1
x_60 -0
x_61 -0
x_62 -0
x_63 -0
x_64 -0
x_65 -0
x_66 -0
x_67 -0
x_68 -0
x_69 -0
x_610 -0
x_611 -0
x_612 -0
x_613 1
x_614 -0
x_70 -0
x_71 -0
x_72 -0
x_73 -0
x_74 -0
x_75 -0
x_76 1
x_77 -0
x_78 -0
x_79 -0
x_710 -0
x_711 -0
x_712 -0
x_713 -0
x_714 -0
x_80 -0


#### Step 8 Alternate: Print the solution (The Easy To Read Way)

In [10]:
print('\nObjective Function Value: %g' %m.objVal) # prints objective function value
print('')
print('Solution:')
print('')
print('total groups unseated : %s ' %z.x)
print('')

for j in range(J):
    print('For row %s:' %j)
    print('total empty seats: %s' %y[j].x)
    
    for i in range(I):
        print(' Group %s sits %g' %(i,x[i,j].x)) # prints out number of operation j in each room i

   


Objective Function Value: 576

Solution:

total groups unseated : 0.0 

For row 0:
total empty seats: 13.0
 Group 0 sits -0
 Group 1 sits -0
 Group 2 sits -0
 Group 3 sits -0
 Group 4 sits -0
 Group 5 sits -0
 Group 6 sits -0
 Group 7 sits -0
 Group 8 sits -0
 Group 9 sits -0
 Group 10 sits -0
 Group 11 sits -0
 Group 12 sits -0
 Group 13 sits -0
 Group 14 sits -0
 Group 15 sits -0
 Group 16 sits -0
 Group 17 sits -0
 Group 18 sits -0
 Group 19 sits -0
 Group 20 sits -0
 Group 21 sits -0
 Group 22 sits -0
 Group 23 sits -0
 Group 24 sits -0
 Group 25 sits -0
 Group 26 sits -0
 Group 27 sits -0
 Group 28 sits -0
 Group 29 sits -0
 Group 30 sits 1
 Group 31 sits -0
 Group 32 sits -0
 Group 33 sits -0
 Group 34 sits -0
For row 1:
total empty seats: -0.0
 Group 0 sits -0
 Group 1 sits -0
 Group 2 sits -0
 Group 3 sits -0
 Group 4 sits -0
 Group 5 sits -0
 Group 6 sits -0
 Group 7 sits -0
 Group 8 sits 1
 Group 9 sits 1
 Group 10 sits -0
 Group 11 sits -0
 Group 12 sits -0
 Group 13 sits -

#### Optional Step: Exporting the LP File for Debugging
Do you think that something is wrong with your model? Export it to an LP file and check if it is in the desired shape.

In [11]:
m.write("checkModel.lp")